In [1]:
# Get input
input_test = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

#print(input_test)

inpute = open("./input.txt").read()

In [2]:
# Imports
# import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
import multiprocess as mp
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor

## Part 1

In [3]:
# Split on new lines
lines = inpute.split('\n')

In [4]:
# Initialize a dictionary to store each section data
seeds_nb = []
section_data = {}

In [5]:
# Function to process a section and extract numbers
def process_section(section_lines):
    section_data = []
    for line in section_lines:
        numbers = [int(num) for num in line.split() if num.isdigit()]
        section_data.append(numbers)
    return section_data

In [6]:
# Iterate through each line and process the corresponding section
current_section = None
for line in lines:
    if line:
        if ':' in line:
            current_section = line.split(':')[0].strip()
            if current_section == 'seeds':
                seeds_nb.extend(int(num) for num in line.split(':')[1].strip().split(' ') if num.isdigit())
                #section_data[current_section] = [int(num) for num in line.split(':')[1].strip().split(' ') if num.isdigit()]
            else:
                section_data[current_section] = []
        else:
            section_data[current_section].extend(process_section([line]))

#print(list(seeds_nb))
#for section, data in section_data.items():
#    print(f"{section}: {data}")

In [7]:
'''
# Create a dict for seeds location values
def compute_values(seeds_nb):
    seeds_map = {}
    for seed in seeds_nb:
        seeds_map[seed] = seed
    
        for key, value in section_data.items():
            for line in section_data[key]:
                    if seeds_map[seed] in range(line[1], line[1]+line[2]):
                        seeds_map[seed] = seeds_map[seed]+(line[0]-line[1])
                        break

    return seeds_map
'''

'\n# Create a dict for seeds location values\ndef compute_values(seeds_nb):\n    seeds_map = {}\n    for seed in seeds_nb:\n        seeds_map[seed] = seed\n    \n        for key, value in section_data.items():\n            for line in section_data[key]:\n                    if seeds_map[seed] in range(line[1], line[1]+line[2]):\n                        seeds_map[seed] = seeds_map[seed]+(line[0]-line[1])\n                        break\n\n    return seeds_map\n'

In [8]:
seeds_map = {}

for seed in seeds_nb:
    seeds_map[seed] = seed

In [9]:
def process_seed(seed):
    seeds_map[seed] = seed
    
    for key, value in section_data.items():
        for line in section_data[key]:
                if seeds_map[seed] in range(line[1], line[1]+line[2]):
                    seeds_map[seed] = seeds_map[seed]+(line[0]-line[1])
                    break

    return seeds_map[seed]

In [10]:
with mp.Pool(5) as pool:
    final_seeds = pool.map(process_seed, seeds_nb)

In [11]:
min(final_seeds)

322500873

## Part 2

### Lists + multiprocessing
Time logs:
- range/100000 + Pool(100): 6.95 s ± 65.5 ms per loop
- range/100000 + Pool(1000): 20.1 s ± 568 ms per loop

In [35]:
seeds_nbp = np.array(seeds_nb)

In [37]:
%%timeit
seeds_new = np.array([])

for i in range(0, len(seeds_nbp)-1, 2):
    for j in range(seeds_nbp[i], seeds_nbp[i]+seeds_nbp[i+1], 100000):
        #seeds_new.append(j)
        seeds_new = np.concatenate((seeds_new, np.array([j])))

66.4 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%%timeit
seeds_map = {}

for seed in seeds_new:
    seeds_map[seed] = seed

151 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%%timeit
with mp.Pool(15) as pool:
    final_seeds_2 = pool.map(process_seed, seeds_new)

6.81 s ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
#%%timeit
min(final_seeds_2)

108956286

### Torch tensors